# Homework 2 - Convolutional Neural Nets

In this homework, we will be working with google [colab](https://colab.research.google.com/). Google colab allows you to run a jupyter notebook on google servers using a GPU or TPU. To enable GPU support, make sure to press Runtime -> Change Runtime Type -> GPU. 

## Cats vs dogs classification

To learn about and experiment with convolutional neural nets we will be working on a problem of great importance in computer vision - classifying images of cats and dogs.

The problem is so important that there's even an easter egg in colab: go to Tools -> Settings -> Miscellaneous and enable 'Corgi mode' and 'Kitty mode' to get more cats and dogs to classify when you're tired of coding.



### Getting the data

To get started with the classification, we first need to download and unpack the dataset (note that in jupyter notebooks commands starting with `!` are executed in bash, not in python):

In [ ]:
! wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

In [ ]:
! unzip cats_and_dogs_filtered.zip 

This dataset contains two directories, `train` and `validation`. Both in turn contain two directories with images: `cats` and `dogs`. In `train` we have 1000 images of cats, and another 1000 images of dogs. For `validation`, we have 500 images of each class. Our goal is to implement and train a convolutional neural net to classify these images, i.e. given an image from this dataset, tell if it contains a cat or a dog.



In [ ]:
! echo 'Training cats examples:' $(find cats_and_dogs_filtered/train/cats -type f | wc -l)
! echo 'Training dogs examples:' $(find cats_and_dogs_filtered/train/dogs -type f | wc -l)
! echo 'Validation cats examples:' $(find cats_and_dogs_filtered/validation/cats -type f | wc -l)
! echo 'Validation dogs examples:' $(find cats_and_dogs_filtered/validation/dogs -type f | wc -l)

### Loading the data
 Now that we have the data on our disk, we need to load it so that we can use it to train our model. In Pytorch ecosystem, we use `Dataset` class, documentation for which can be found [here](https://pytorch.org/docs/stable/data.html). 

 In the case of computer vision, the datasets with the folder structure 'label_name/image_file' are very common, and to process those there's already a class `torchvision.datasets.ImageFolder` (documented [here](https://pytorch.org/vision/0.8/datasets.html)). Torchvision is a Pytorch library with many commonly used tools in computer vision.

 Another thing we need from Torchvision library is transforms ([documentation](https://pytorch.org/docs/stable/torchvision/transforms.html)). In computer vision, we very often want to transform the images in certain ways. The most common is normalization. Others include flipping, changing saturation, hue, contrast, rotation, and blurring. 

 Below, we create a training, validation and test sets. We use a few transforms for augmentation on the training set, but we don't use anything but resize and normalization for validation and test. 

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image # PIL is a library to process images

# These numbers are mean and std values for channels of natural images. 
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Inverse transformation: needed for plotting.
unnormalize = transforms.Normalize(
   mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
   std=[1/0.229, 1/0.224, 1/0.225]
)

train_transforms = transforms.Compose([
                                    transforms.Resize((256, 256)),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ColorJitter(hue=.1, saturation=.1, contrast=.1),
                                    transforms.RandomRotation(20, resample=Image.BILINEAR),
                                    transforms.GaussianBlur(7, sigma=(0.1, 1.0)),
                                    transforms.ToTensor(),  # convert PIL to Pytorch Tensor
                                    normalize,
                                ])

validation_transforms = transforms.Compose([
                                    transforms.Resize((256, 256)),
                                    transforms.ToTensor(), 
                                    normalize,
                                ])

train_dataset = torchvision.datasets.ImageFolder('./cats_and_dogs_filtered/train', transform=train_transforms)
print(len(train_dataset))
validation_dataset, test_dataset = torch.utils.data.random_split(torchvision.datasets.ImageFolder('./cats_and_dogs_filtered/validation', transform=validation_transforms), [500, 500], generator=torch.Generator().manual_seed(42))

2000


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1201: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


Let's see what one of the images in the dataset looks like (you can run this cell multiple times to see the effects of different augmentations):

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 200 # change dpi to make plots bigger

def show_normalized_image(img, title=None):
  plt.imshow(unnormalize(img).detach().cpu().permute(1, 2, 0))
  plt.title(title)
  plt.axis('off')

show_normalized_image(train_dataset[10][0])
print(train_dataset[10][1])      # 0: cat
print(train_dataset[1110][1])    # 1: dog
train_dataset[10][0].shape       # torch.Size([3, 256, 256])

### Creating the model

Now is the time to create a model. All models in Pytorch are subclassing `torch.nn.Module`, and have to implement `__init__` and `forward` methods. 

Below we provide a simple model skeleton, which you need to expand. The places to put your code are marked with `TODO`. Here, we ask you to implement a convolutional neural network containing the following elements:

* Convolutional layers (at least two)
* Batch Norm
* Non-linearity
* Pooling layers
* A residual connection similar to that of Res-Net
* A fully connected layer

For some examples of how to implement Pytorch models, please refer to our lab notebooks, such as [this one](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/06-convnet.ipynb).

In [ ]:
from torch import nn

class CNN(torch.nn.Module):

  def __init__(self):
    super().__init__()
    self.conv0 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1)
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=4)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
    self.conv4 = nn.Conv2d(128, 256, kernel_size=3)
    self.conv5 = nn.Conv2d(256, 256, kernel_size=3)
    self.conv06 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.conv6 = nn.Conv2d(256, 512, kernel_size=3)

    self.bn1 = nn.BatchNorm2d(32)
    self.bn2 = nn.BatchNorm2d(64)
    self.bn3 = nn.BatchNorm2d(128)
    self.bn4 = nn.BatchNorm2d(256)
    self.bn5 = nn.BatchNorm2d(256)
    self.bn6 = nn.BatchNorm2d(512)

    self.fc1 = nn.Linear(512*2*2,50)
    self.fc2 = nn.Linear(50,20)
    self.fc3 = nn.Linear(20,2)
  def forward(self, x):
    residual = x
    x = self.conv0(x)
    x += residual

    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2) #[64*32*127*127]
    
    x = self.bn1(x)
    x = F.relu(x)   #[64*32*127*127]

    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2) #[64*32*62*62]

    x = self.bn2(x)
    x = F.relu(x)   #[64*32*62*62]

    x = self.conv3(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2) #[64*64*30*30]
    
    x = self.bn3(x)
    x = F.relu(x)   #[64*64*30*30]

    x = self.conv4(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2) #[64*64*14*14]

    x = self.bn4(x)
    x = F.relu(x)   #[64*64*14*14]

    x = self.conv5(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2) #[64*128*6*6]
    
    x = self.bn5(x)
    x = F.relu(x)   #[64*128*6*6]

    residual = x
    x = self.conv06(x)
    x += residual

    x = self.conv6(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2) #[64*128*2*2]

    x = self.bn6(x)
    x = F.relu(x)   #[64*512*2*2]

    x = x.view(-1, 512*2*2)  # [64*(512*2*2)]
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)  # [64*50]
    x = F.relu(x)
    x = self.fc3(x)  # [64*2]
#     x = F.log_softmax(x, dim=1)
    return x

### Training the model

Now we train the model on the dataset. Again, we're providing you with the skeleton with some parts marked as `TODO` to be filled by you.

In [ ]:
from tqdm.notebook import tqdm

def get_loss_and_correct(model, batch, criterion, device):
  # Implement forward pass and loss calculation for one batch.
  # Remember to move the batch to device.

  # Return a tuple:
  # - loss for the batch (Tensor)
  # - number of correctly classified examples in the batch (Tensor)

  x = batch[0].to(device)   #torch.Size([64, 3, 256, 256])
  y = batch[1].to(device)   #torch.Size([64])
  model.to(device)
  # Training 
  # Feed forward to get the logits
  y_hat = model(x)

  # Compute the loss and accuracy
  loss = criterion(y_hat, y)
  score, predicted = torch.max(y_hat, 1)
  acc = (y == predicted).sum()
  
  return (loss, acc)


def step(loss, optimizer):
  # Implement backward pass and update.
  
  # zero the gradients before running
  # the backward pass.
  
  # display.clear_output(wait=True)
  optimizer.zero_grad()
  
  # Backward pass to compute the gradient
  # of loss w.r.t our learnable params. 
  loss.backward()
  
  # Update params
  optimizer.step()

N_EPOCHS = 10
BATCH_SIZE = 64

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=BATCH_SIZE, num_workers=4)
model = CNN()

# The negative log likelihood loss. It is useful to train a classification problem with C classes.
# criterion = nn.NLLLoss()
criterion = torch.nn.CrossEntropyLoss()

learning_rate = 1e-3
lambda_l2 = 1e-5

# stochastic gradient descent for our parameter updates
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # 61%

# # ADAM for our parameter updates
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # 73%

# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5) # 70%


model.train()

if torch.cuda.is_available():
  model = model.cuda()
  criterion = criterion.cuda()
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []

pbar = tqdm(range(N_EPOCHS))

for i in pbar:
  total_train_loss = 0.0
  total_train_correct = 0.0
  total_validation_loss = 0.0
  total_validation_correct = 0.0

  model.train()

  for batch in tqdm(train_dataloader, leave=False):
    loss, correct = get_loss_and_correct(model, batch, criterion, device)
    step(loss, optimizer)
    total_train_loss += loss.item()
    total_train_correct += correct.item()

  with torch.no_grad():
    for batch in validation_dataloader:
      loss, correct = get_loss_and_correct(model, batch, criterion, device)
      total_validation_loss += loss.item()
      total_validation_correct += correct.item()

  mean_train_loss = total_train_loss / len(train_dataset)
  train_accuracy = total_train_correct / len(train_dataset)

  mean_validation_loss = total_validation_loss / len(validation_dataset)
  validation_accuracy = total_validation_correct / len(validation_dataset)

  train_losses.append(mean_train_loss)
  validation_losses.append(mean_validation_loss)

  train_accuracies.append(train_accuracy)
  validation_accuracies.append(validation_accuracy)

  pbar.set_postfix({'train_loss': mean_train_loss, 'validation_loss': mean_validation_loss, 'train_accuracy': train_accuracy, 'validation_accuracy': validation_accuracy})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from torchsummary import summary
summary(model, input_size=(3,256,256))

Now that the model is trained, we want to visualize the training and validation losses and accuracies:

In [ ]:
plt.figure(dpi=200)

plt.subplot(121)
plt.plot(train_losses, label='train')
plt.plot(validation_losses, label='validation')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Losses')
plt.legend()

plt.subplot(122)
plt.plot(train_accuracies, label='train')
plt.plot(validation_accuracies, label='validation')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.ylim(0, 1)
plt.title('Accuracies')

plt.tight_layout()

Now, change your model to achieve at least 75% accuracy on validation set. You can change the model you've implemented, the optimizer, and the augmentations. 

Looking at the loss and accuracy plots, can you see if your model overfits the trainig set? Why?

Answer:
`The model does not overfit. First I set weight_decay = 1e-5 for optimizer to avoid overfitting. Second, as shown in the above plots, the train_dataset has the similar accuracy curve of that of the validation_dataset.`

### Testing the model

Now, use the `test_dataset` to get the final accuracy of your model. Visualize some correctly and incorrectly classified examples.

In [ ]:
# TODO
# 1. Calculate and show the test_dataset accuracy of your model.
# 2. Visualize some correctly and incorrectly classified examples.
def test(model, batch, device, printExample):
  x = batch[0].to(device)   #torch.Size([64, 3, 256, 256])
  y = batch[1].to(device)   #torch.Size([64])
  model.to(device)

  output = model(x)

  # Compute the accuracy
  score, predicted = torch.max(output, 1)
  acc = (y == predicted).sum()

  if printExample:
    corr = False
    incorr = False
    for i in range(64):
      if not corr and y[i] == predicted[i]:
        show_normalized_image(img=x[i], title='Correctly classified example')
        plt.show()
        corr = True
      if not incorr and y[i] != predicted[i]:
        show_normalized_image(img=x[i], title='Incorrectly classified example')
        plt.show()
        incorr = True
      if corr and incorr:
        break
  
  return acc

model.eval()
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=4)
total_test_correct = 0.0
i = 0
num_to_print = 3

with torch.no_grad():
  for batch in test_dataloader:
    correct = test(model, batch, device, i < num_to_print)
    total_test_correct += correct.item()
    i += 1

test_accuracy = total_test_correct / len(test_dataset)
print('Test_accuracy is {}.'.format(test_accuracy))


### Visualizing filters

In this part, we are going to visualize the output of one of the convolutional layers to see what features they focus on.

First, let's get some image.

In [ ]:
image = validation_dataset[10][0]
show_normalized_image(image)

Now, we are going to 'clip' our model at different points to get different intermediate representation. 
Clip your model at two or three different points and plot the filters output.

In order to clip the model, you can use `model.children()` method. For example, to get output only after the first 4 layers, you can do:

```
clipped = nn.Sequential(
    *list(model.children())[:4]
)
intermediate_output = clipped(input)
```



In [ ]:
import math

def plot_intermediate_output(result, title):
  """ Plots the intermediate output of shape
      N_FILTERS x H x W
  """
  n_filters = result.shape[1]
  N = int(math.sqrt(n_filters))
  M = (n_filters + N - 1) // N
  assert N * M >= n_filters
  
  fig, axs = plt.subplots(N, M)
  fig.suptitle(title)
  
  for i in range(N):
    for j in range(M):
      if i*N + j < n_filters:
        axs[i][j].imshow(result[0, i*N + j].cpu().detach())
        axs[i][j].axis('off')

# TODO: 
# pick a few intermediate representations from your network and plot them using 
# the provided function.
def get_intermediate_output(image, ithLayer):
  img = torch.unsqueeze(image.to(device),0)
  clipped = nn.Sequential(
    *list(model.children())[:ithLayer]
  )
  intermediate_output = clipped(img)
  print("Output after the first {} layers".format(ithLayer))
  print(intermediate_output.shape)
  print(intermediate_output)
  plot_intermediate_output(intermediate_output, "Output after the first {} layers".format(ithLayer))

get_intermediate_output(image,3)
get_intermediate_output(image,6)

In [ ]:
child_pointer = 0
for child in model.children():
  print(child_pointer, child)
  
  while child_pointer <= 7:
    print("conv{}".format(child_pointer))
    para1,para2 = child.parameters()
    print(para1)
    print(para2)
    # plot_intermediate_output(para1[0], "Output after the first {} layers".format(child_pointer))
    break
  break
  child_pointer += 1

What can you say about those filters? What features are they focusing on?

Anwer: `In the initial steps, there are many activations on the edges and textures within the image, especially the outline of the animal. The filters are focused on extract the shape of the animals from the background, after removing the noise caused by color and light difference. Later on, the filters try to find the inner difference within the animal, so that cats and dogs can be classified.`